In [1]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

In [2]:

llm = ChatOllama(
    model = "llama3.1",
    #model = "gemma",
    base_url = "http://localhost:11434")


In [3]:
## GENERATE

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generate = prompt | llm
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

The Little Prince, written by Antoine de Saint-Exupéry, has been a timeless classic since its publication in 1943. Despite being penned over seven decades ago, this novella continues to captivate children and adults alike with its poignant tale of friendship, love, and the human condition. The themes explored in The Little Prince are remarkably relevant to modern childhood, making it an essential read for young people today.

One of the primary reasons why The Little Prince remains so pertinent is its exploration of the complexities of growing up. In a world that often values material possessions and external validation, the novella's protagonist, the Little Prince, teaches children about the importance of inner wisdom, emotional intelligence, and genuine connections with others. As children navigate their own experiences of growth, The Little Prince reminds them that true happiness comes from nurturing meaningful relationships and pursuing one's passions, rather than merely accumulati

In [ ]:
## REFLEXION

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content


**Grade:** B+ (87%)

**

In [ ]:
##REPEAT    
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")


In [ ]:
##### USING LANGGRAPH

In [ ]:
import asyncio
from langgraph.graph import END, MessageGraph
from typing import Sequence
from langchain_core.messages import HumanMessage
from typing import List



from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant researcher tasked with researching on a variety of topics in a short summary of 5 paragraphs."
            " Generate the best research possible as per user request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)



generate = prompt | llm

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a senior researcher"
            " Provide detailed recommendations, including requests for length, depth, style, etc."
            " to an asistant researcher to help improve this researches",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflect = reflection_prompt | llm


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}

    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})

    # this will be treated as a feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 2:
        # End after 2 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()


import nest_asyncio
nest_asyncio.apply()

async def stream_responses():
    async for event in graph.astream(
        [
            HumanMessage(
                content="Research on climate change"
            )
        ],
    ):
        print(event)
        print("---")


asyncio.run(stream_responses())   
 

